In [2]:
import os
from PIL import Image
from tqdm import tqdm
import pickle

import pandas as pd
from glob import glob
import pillow_heif
import cv2

from facenet_pytorch import MTCNN
import torch
import torchvision

import csv
import json
import numpy as np
from keras_facenet import FaceNet
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.models import load_model

c:\Users\Senne\AI_Frameworks\Project\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
test_data = glob('testset/testset/*.jpg')
source_folder = "Testset/Testset"
output_folder = "Testset/Testset"

def list_extensions(folder_path):
    extensions = set()
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        if os.path.isfile(file_path):
            _, extension = os.path.splitext(filename)
            extensions.add(extension.lower())
    return extensions

def convert_to_jpg(file_path, source_folder):
    try:
        _, extension = os.path.splitext(file_path)
        extension = extension.lower()

        if extension == ".heic":
            heif_file = pillow_heif.open_heif(file_path)
            image = Image.frombytes(
                heif_file.mode, heif_file.size, heif_file.data, "raw"
            )
        else:
            image = Image.open(file_path)

        image = image.convert("RGB")

        base_name = os.path.basename(file_path)
        new_file_path = f"{source_folder}{os.path.splitext(base_name)[0]}.jpg"
        os.remove(file_path)
        image.save(new_file_path, "JPEG")
        print(f"Converted {file_path} to {new_file_path}")

        return new_file_path
    except Exception as e:
        print(f"Failed to convert {file_path}: {e}")
        return None

def convert_images_from_df(df, source_folder):
    new_images = []
    for _, row in df.iterrows():
        file_path = row["File Path"]
        converted_path = convert_to_jpg(file_path, source_folder)
        if converted_path:
            new_images.append(converted_path)
    return new_images


def extract_frame_from_mp4(video_path, output_folder, frame_number=0):
    try:
        video = cv2.VideoCapture(video_path)
        
        video.set(cv2.CAP_PROP_POS_FRAMES, frame_number)
        success, frame = video.read()
        video.release()

        if not success:
            print(f"Failed to extract frame from {video_path}")
            return None
        
        base_name = os.path.basename(video_path)
        output_file = os.path.join(output_folder, f"{os.path.splitext(base_name)[0]}.jpg")
        
        os.remove(video_path)
        cv2.imwrite(output_file, frame)
        print(f"Extracted frame from {video_path} and saved to {output_file}")
        return output_file
    except Exception as e:
        print(f"Error processing {video_path}: {e}")
        return None

def process_mp4_to_jpg(mp4_df, output_folder):
    jpg_files = []
    for _, row in mp4_df.iterrows():
        video_path = row["File Path"]
        jpg_path = extract_frame_from_mp4(video_path, output_folder)
        if jpg_path:
            jpg_files.append(jpg_path)
    return jpg_files

def list_non_jpg_images(folder_path, image_extensions):
    non_jpg_files = []
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        if os.path.isfile(file_path):
            _, extension = os.path.splitext(filename)
            if extension.lower() in image_extensions:
                non_jpg_files.append(file_path)
    return non_jpg_files

extensions_in_folder = list_extensions(source_folder)

extensions_in_folder.discard(".jpg")
if ".mp4" in extensions_in_folder:
    mp4_Extension = ".mp4"

print("Extensions found in the folder:")
for ext in sorted(extensions_in_folder):
    print(ext if ext else "No extension (e.g., hidden or extensionless files)")

non_jpg_images = list_non_jpg_images(source_folder, extensions_in_folder)
img_not_jpg_df = pd.DataFrame(non_jpg_images, columns=["File Path"])

if ".mp4" in extensions_in_folder:
    mp4_images = list_non_jpg_images(source_folder, mp4_Extension)
    if mp4_images:
        mp4_img_df = pd.DataFrame(mp4_images, columns=["File Path"])
    
convert_images_from_df(img_not_jpg_df, output_folder)
if ".mp4" in extensions_in_folder:
    jpg_files = process_mp4_to_jpg(mp4_img_df, output_folder)
    mp4_to_jpg_df = pd.DataFrame(jpg_files, columns=["image"])

Extensions found in the folder:


In [9]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"PyTorch version: {torch.__version__}")
print(f"Torchvision version: {torchvision.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"CUDA device: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'None'}")
print(f"Using device: {device}")
mtcnn = MTCNN(keep_all=True, device=device)

PyTorch version: 2.2.2+cu118
Torchvision version: 0.17.2+cu118
CUDA available: True
CUDA device: NVIDIA GeForce RTX 3050 Ti Laptop GPU
Using device: cuda


In [10]:
def detect_faces_mtcnn(img, img_index, cropped_faces):
    try:
        correct_rotation = 0
        highest_prob = 0.0
        confidence_threshold = 0.82
        length = 4
        horizontal_preffered_dimensions = (1008, 756)
        vertical_preffered_dimensions = (756, 1008)
        valid_boxes = []
        best_probs = []
        found_landmarks  = []
        #print(img.shape)
        for i in range(length):
            empty_box = False
            degrees = 90 * i
            rotated_img = img

            if degrees == 90:
                rotated_img = cv2.rotate(img, cv2.ROTATE_90_CLOCKWISE)
            elif degrees == 180:
                rotated_img = cv2.rotate(img, cv2.ROTATE_180)
            elif degrees == 270:
                rotated_img = cv2.rotate(img, cv2.ROTATE_90_COUNTERCLOCKWISE)

            height, width, _ = rotated_img.shape

            if width > height: 
                scale = min(horizontal_preffered_dimensions[0] / width, horizontal_preffered_dimensions[1] / height)
            else:
                scale = min(vertical_preffered_dimensions[0] / width, vertical_preffered_dimensions[1] / height)

            new_width = int(width * scale)
            new_height = int(height * scale)

            resized_img = cv2.resize(rotated_img, (new_width, new_height))
            #resized_img = cv2.resize(rotated_img, (width, height))
            #print(resized_img.shape)
                
            boxes, probs, landmarks = mtcnn.detect(resized_img, landmarks=True)

            if boxes is None or probs is None or landmarks is None:
                empty_box = True
                boxes, landmarks = [], []

            #print(f"Image index {img_index}: Detection probabilities - {probs}")
            rotated_img = cv2.cvtColor(rotated_img, cv2.COLOR_BGR2RGB)

            if not empty_box:
                filtered = [
                    (box, prob, landmark)
                    for box, prob, landmark in zip(boxes, probs, landmarks)
                    if prob >= confidence_threshold
                    #if Valid_Landmarks(box, landmark)
                ]
                boxes, probs, landmarks = zip(*filtered) if filtered else ([], [], [])
                boxes = sorted(boxes, key=lambda box: box[0])

                for prob in probs:
                    if prob > highest_prob:
                        highest_prob = prob
                        best_probs = probs
                        valid_boxes = boxes
                        found_landmarks = landmarks
                        correct_rotation = degrees

        if correct_rotation == 90:
            img = cv2.rotate(img, cv2.ROTATE_90_CLOCKWISE)
        elif correct_rotation == 180:
            img = cv2.rotate(img, cv2.ROTATE_180)
        elif correct_rotation == 270:
            img = cv2.rotate(img, cv2.ROTATE_90_COUNTERCLOCKWISE)
            
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        for box, landmark, prob in zip(valid_boxes, found_landmarks, best_probs):
            #print(f"Image index {img_index}: Detection probabilities - {prob}")    
            
            height, width, _ = img.shape
            x1, y1, x2, y2 = [int(coord) for coord in box]
            
            x1 = int(x1 / scale)
            y1 = int(y1 / scale)
            x2 = int(x2 / scale)
            y2 = int(y2 / scale)

            x1 = max(0, min(x1, width - 1))
            y1 = max(0, min(y1, height - 1))
            x2 = max(0, min(x2, width - 1))
            y2 = max(0, min(y2, height - 1))

            cropped_face = img[y1:y2, x1:x2]
            cropped_faces.append(cropped_face)

            #cropped_face = cv2.cvtColor(cropped_face, cv2.COLOR_BGR2RGB)

            #cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)

            #print(f"Face landmarks: {landmark}")
            for point in landmark:
                px, py = int(point[0]), int(point[1])
                #cv2.circle(img, (px, py), 5, (255, 0, 0), -1)  # Draw landmarks

            #plt.imshow(cropped_face)
            #plt.axis("off")
            #plt.show()

        #img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        #plt.imshow(img)
        #plt.axis("off")
        #plt.show()

        Cropped_FilePath = "CroppedTestImg"
        os.makedirs(Cropped_FilePath, exist_ok=True)

        return len(valid_boxes)
    except Exception as e:
        print(f"Error detecting faces in image index {img_index}: {e}")
        return 0

In [11]:
start = 0
Cropped_FilePath = "CroppedTestImg"
CroppedImg_filepaths = []
bad_detect = pd.DataFrame()

bad = 0
good = 0

length = len(test_data)
for i in range(length):

    cropped_faces = []
    position = i + start
    img = cv2.imread(test_data[position])
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    detected = detect_faces_mtcnn(img, position, cropped_faces)

    img_name = os.path.basename(test_data[position])
    img_name = os.path.splitext(img_name)[0]

    for d in range(detected):
        Img_FilePath = os.path.join(Cropped_FilePath, f"{img_name}_{d}.jpg")
        CroppedImg_filepaths.append(Img_FilePath)
        cv2.imwrite(Img_FilePath, cropped_faces[d])
        good += 1

    if detected == 0:
        bad_detect = pd.concat([bad_detect, pd.DataFrame({
            'image': [test_data[position]],
            'guess': ['nothing']
        })], ignore_index=True)
        bad += 1

    print(f"Processed {i+1}/{length} images")

print(f"faces found: {good}")
print(f"None faces found: {bad}")

Processed 1/99 images
Processed 2/99 images
Processed 3/99 images
Processed 4/99 images
Processed 5/99 images
Processed 6/99 images
Processed 7/99 images
Processed 8/99 images
Processed 9/99 images
Processed 10/99 images
Processed 11/99 images
Processed 12/99 images
Processed 13/99 images
Processed 14/99 images
Processed 15/99 images
Processed 16/99 images
Processed 17/99 images
Processed 18/99 images
Processed 19/99 images
Processed 20/99 images
Processed 21/99 images
Processed 22/99 images
Processed 23/99 images
Processed 24/99 images
Processed 25/99 images
Processed 26/99 images
Processed 27/99 images
Processed 28/99 images
Processed 29/99 images
Processed 30/99 images
Processed 31/99 images
Processed 32/99 images
Processed 33/99 images
Processed 34/99 images
Processed 35/99 images
Processed 36/99 images
Processed 37/99 images
Processed 38/99 images
Processed 39/99 images
Processed 40/99 images
Processed 41/99 images
Processed 42/99 images
Processed 43/99 images
Processed 44/99 imag

In [2]:
def resize_with_padding(input_dir, output_dir, target_size=(160, 160)):
    os.makedirs(output_dir, exist_ok=True)

    for root, _, files in os.walk(input_dir):
        for file in tqdm(files, desc=f"Processing images in {root}"):
            try:
                input_path = os.path.join(root, file)
                output_subdir = os.path.join(output_dir, os.path.relpath(root, input_dir))
                os.makedirs(output_subdir, exist_ok=True)

                output_path = os.path.join(output_subdir, file)

                with Image.open(input_path) as img:
                    img = img.convert("RGB")

                    if img.width < target_size[0] and img.height < target_size[1]:
                        scale_factor = min(
                            target_size[0] / img.width,
                            target_size[1] / img.height
                        )
                        new_width = int(img.width * scale_factor)
                        new_height = int(img.height * scale_factor)
                        img = img.resize((new_width, new_height), Image.Resampling.LANCZOS)

                    else:
                        img.thumbnail(target_size, Image.Resampling.LANCZOS)

                    new_img = Image.new("RGB", target_size, (0, 0, 0))

                    offset_x = (target_size[0] - img.width) // 2
                    offset_y = (target_size[1] - img.height) // 2
                    new_img.paste(img, (offset_x, offset_y))

                    new_img.save(output_path)

            except Exception as e:
                print(f"Error processing file {file}: {e}")

In [3]:
input_dir = "CroppedTestImg"
output_dir = "CroppedTestImg_resized_160"
resize_with_padding(input_dir, output_dir)

Processing images in CroppedTestImg: 100%|██████████| 178/178 [00:01<00:00, 104.76it/s]


In [7]:
facenet = FaceNet()
base_model = facenet.model

with open("Labels_faces.pickle", 'rb') as f:
    class_labels = pickle.load(f)
    class_labels = {n: i for i, n in class_labels.items()}
print(f"Class labels: {class_labels}")

num_classes = len(class_labels)

x = base_model.output
x = Flatten()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
output = Dense(num_classes, activation='softmax')(x)

fine_tuned_model = Model(inputs=base_model.input, outputs=output)

fine_tuned_model.load_weights('fine_tuned_model_weights.h5')

testset_dir = "CroppedTestImg_resized_160"
original_testset_dir = "testset/testset"
output_csv = "FinalResults_UT_54_160.csv"

def preprocess_image(img_path, target_size=(160, 160)):
    """
    Preprocess a single image.
    """
    img = load_img(img_path, target_size=target_size)
    img_array = img_to_array(img) / 255.0
    return np.expand_dims(img_array, axis=0)

results = {}
Grouping_number = []
Current_number = 0

for file_name in os.listdir(testset_dir):
    prefix = file_name.split('_')[0]
    img_number = prefix.lstrip("0")
    if (img_number not in Grouping_number):
        Grouping_number.append(img_number)
        results[img_number] = []
        Current_number = img_number
        #print("new number")

    image_path = os.path.join(testset_dir, file_name)
    #print(image_path)

    if not os.path.isfile(image_path):
        #print("not a file")
        continue

    try:
        preprocessed_img = preprocess_image(image_path)

        predictions = fine_tuned_model.predict(preprocessed_img, verbose=0)
        predicted_class = np.argmax(predictions)
        predicted_label = class_labels[predicted_class].lower()

        results[Current_number].append(predicted_label)

    except Exception as e:
        print(f"Error processing image {image_path}: {e}")
        results[Current_number].append("error")

#print(results)

all_image_numbers = sorted({int(file.split('.')[0]) for file in os.listdir(original_testset_dir) if file.lower().endswith(('.jpg'))})
#print(all_image_numbers)
predicted_image_numbers = sorted(results.keys(), key=int)
predicted_image_numbers = [int(num) for num in predicted_image_numbers]
#print(predicted_image_numbers)
missing_image_numbers = [num for num in all_image_numbers if num not in predicted_image_numbers]
print(missing_image_numbers)

for missing_number in sorted(missing_image_numbers):
    results[missing_number] = ["nothing"]

with open(output_csv, mode="w", newline="") as csv_file:
    csv_writer = csv.writer(csv_file)
    csv_writer.writerow(["image", "label_name"])

    for image_number in sorted(results.keys(), key=int):
        labels_in_order = ";".join(results[image_number])
        csv_writer.writerow([image_number, labels_in_order])

print(f"Predictions saved to {output_csv}")

Class labels: {0: 'Akif', 1: 'Alper', 2: 'Bart', 3: 'Daiane', 4: 'Florian', 5: 'Konrad', 6: 'Lasse', 7: 'Matthias', 8: 'Michiel', 9: 'Nelli', 10: 'Raul', 11: 'Senne', 12: 'Seppe', 13: 'Youssef'}
[45, 427, 513, 770]
Predictions saved to FinalResults_UT_54_160.csv
